# Generating Qubit Hamiltonians

In [1]:
from utility import *
from functools import reduce

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [2]:
# Identity 
I = np.asarray([[1.,0.],[0.,1.]])
# Pauli X
X = np.asarray([[0.,1.],[1.,0.]])
# Pauli Y
Y = np.asarray([[0.,-1j],[1j,0.]])
# Pauli Z
Z = np.asarray([[1, 0], [0, -1]])

In [3]:
def operatorfromstring(pauli_string, max_orbitals):
    """
    Generate a many-body operator from a list of single-qubit 
    pauli matrices.
    ---------------------------------------------------------
    Input    pauli_string: str , (ex: [X,Z,X,Y,...])
    
    Output   pauli_op: np.array, shape = [2**N,2**N]
    """
    op_list = []
    eye_num = 0
    
    for k in range(len(pauli_string)):
        if (pauli_string[k][1] == 'X'):
            op_list.append(X)
        elif (pauli_string[k][1] == 'Y'):
            op_list.append(Y)
        elif (pauli_string[k][1] == 'Z'):
            op_list.append(Z)
        else:
            op_list.append(I)
            
    if not op_list:
            op_list.append(I)
            
    if len(op_list) < max_orbitals:
        eye_num = max_orbitals - len(op_list)
        for e in range(eye_num):
            op_list.append(I)         

    return reduce(np.kron,op_list)

def hamiltonianQ(pauli_list, interactions, max_orbitals):
    """
    Generate a many-body hamiltonian
    ---------------------------------------------------------
    Input    pauli_list: list of pauli strings
             interactions: list of interaction strengths
    
    Output   hamiltonian: np.array, shape = [2**N,2**N]
    """
    N = max_orbitals
    hamiltonian = np.zeros((1<<N,1<<N),dtype=complex)
    for i,pauli in enumerate(pauli_list):
        
        hamiltonian += interactions[i]*operatorfromstring(pauli, N)
    return hamiltonian

def eigensolve(hamiltonian):
    """
    Compute ground state energy and wavefunction
    """
    (eigenvalues,eigenstates) = np.linalg.eigh(hamiltonian)
    return eigenvalues[0],eigenstates[:,0]

# H2

In [4]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2)

-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099508 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [5]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841725 [Z0]


We can verify that this new Hamiltonian indeed includes the ground state. 

In [6]:
print("The ground state energy:")
obtain_PES('h2', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
hamiltonian = taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf)
pauli_list = []
interaction_list = []
max_orbitals = 0
#Generate pauli operations list and coefficient list(interactions)
for pauli, interaction in hamiltonian.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)

#Identify Number Max Oribitals    
for x in list(hamiltonian.terms.keys()):
    _max = 0
    for y in x:
        if y[0] > _max:
            _max = y[0]
max_orbitals = _max + 1


h2_matrix = hamiltonianQ(pauli_list, interaction_list, max_orbitals)

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)

print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The ground state energy:
E = -1.1011503301329566 Eh

The eigenvalues in the effective Hamiltonian: 
 [-1.10115033  0.03904763]


# LiH

In [7]:
#########Assumptions made
#Made the assumption based on energy graphs from last notebook
#The hamiltonian outputs the number qubits needed to represent the molecule 
#mapping fermonic states to qubit states --> Qubit representation of hamiltonian
#Jordon-Wigner --> Fermion to Qubit Mapping
qubit_transf = 'jw' # Jordan-Wigner transformations
lih = get_qubit_hamiltonian(mol='lih', geometry=2, basis='sto3g', qubit_transf=qubit_transf)
print(lih)

-4.243807131072307 [] +
-0.0026339817422794706 [X0 X1 Y2 Y3] +
-0.0026512285284603226 [X0 X1 Y2 Z3 Z4 Y5] +
0.0023460572073765486 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0026512285284603226 [X0 X1 X3 X4] +
0.0023460572073765486 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.005497222044565864 [X0 X1 Y4 Y5] +
0.001083012435569457 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
0.001083012435569457 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.002452693385643845 [X0 X1 Y6 Y7] +
-0.002452693385643845 [X0 X1 Y8 Y9] +
-0.002693149673532032 [X0 X1 Y10 Y11] +
0.0026339817422794706 [X0 Y1 Y2 X3] +
0.0026512285284603226 [X0 Y1 Y2 Z3 Z4 X5] +
-0.0023460572073765486 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0026512285284603226 [X0 Y1 Y3 X4] +
0.0023460572073765486 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.005497222044565864 [X0 Y1 Y4 X5] +
-0.001083012435569457 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.001083012435569457 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.002452693385643845 [X0 Y1 Y6 X7] +
0.002452693385643845 [X0 Y1 Y8 X9] +
0.0026931

In [8]:
#########Assumptions made
#
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(lih, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -4.087334375492603 [] +
0.025029550426202134 [X0] +
-0.004655155970512804 [X0 X1 Y2 Y3] +
0.010300171448868658 [X0 X1 Y2 Z3 Z6 Y7] +
0.0006666074318048787 [X0 X1 Z2 X3 Z4 Z5 Z7] +
-0.0005129333695658356 [X0 X1 Z2 Z4 Z5 Z6 X7] +
0.010300171448868658 [X0 X1 X3 X6] +
-0.0006666074318048787 [X0 X1 X3 Z6 Z7] +
0.005404246596177519 [X0 X1 X4] +
0.005404246596177519 [X0 X1 X5] +
-0.03225854086713089 [X0 X1 Y6 Y7] +
0.0005129333695658356 [X0 X1 X7] +
0.0006666074318048787 [X0 Y1 Y2] +
0.004655155970512804 [X0 Y1 Y2 X3] +
-0.0006666074318048787 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-0.010300171448868658 [X0 Y1 Y2 Z3 Z6 X7] +
0.0006666074318048787 [X0 Y1 Z2 Y3 Z4 Z5 Z7] +
-0.0005129333695658356 [X0 Y1 Z2 Z3 Y6] +
0.0005129333695658356 [X0 Y1 Z2 Z4 Z5 Y6 Z7] +
-0.0005129333695658356 [X0 Y1 Z2 Z4 Z5 Z6 Y7] +
0.010300171448868658 [X0 Y1 Y3 X6] +
-0.0006666074318048787 [X0 Y1 Y3 Z6 Z7] +
0.005404246596177519 [X0 Y1 Y4] +
0.005404246596177519 [X0 Y1 Y5] +
0.03225854086713089 [X0 Y1 Y6 

In [9]:
print("The ground state energy:")
obtain_PES('lih', [2], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
hamiltonian = taper_hamiltonian(lih, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf)
pauli_list = []
interaction_list = []
max_orbitals = 0
#Generate pauli operations list and coefficient list(interactions)
for pauli, interaction in hamiltonian.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)

#Identify Number Max Oribitals    
for x in list(hamiltonian.terms.keys()):
    _max = 0
    for y in x:
        if y[0] > _max:
            _max = y[0]
max_orbitals = _max + 1


lih_matrix = hamiltonianQ(pauli_list, interaction_list, max_orbitals)

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(lih_matrix)

print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The ground state energy:
E = -7.861087772411142 Eh

The eigenvalues in the effective Hamiltonian: 
 [-13.4670978  -13.4670978  -13.45750032 -13.45750032 -13.45725375
 -13.45725375 -13.44754254 -13.44754254 -11.56550162 -11.56550162
 -11.55550415 -11.55550415 -11.55197038 -11.55197038 -11.54208961
 -11.54208961 -10.87588441 -10.87588441 -10.86650557 -10.86650557
 -10.86621867 -10.86621867 -10.85683282 -10.85683282  -9.47759628
  -9.47759628  -9.46829203  -9.46829203  -9.46787992  -9.46787992
  -9.45848571  -9.45848571  -8.99512574  -8.99512574  -8.98650537
  -8.98650537  -8.98413469  -8.98413469  -8.97571467  -8.97571467
  -8.97412387  -8.97412387  -8.96632005  -8.96632005  -8.94087147
  -8.94087147  -8.93109587  -8.93109587  -7.72606777  -7.72606777
  -7.71616533  -7.71616533  -7.6685421   -7.6685421   -7.65793882
  -7.65793882  -7.57167818  -7.57167818  -7.56157957  -7.56157957
  -7.55923394  -7.55923394  -7.54924689  -7.54924689  -7.09818851
  -7.09818851  -7.08909174  -7.08909174  -

In [10]:
len(eigvals)

256

# H4

In [12]:
#########Assumptions made
#Chose geometry value from ground state energy aproximation in first notebook 86 something close to a local min

qubit_transf = 'jw' # Jordan-Wigner transformations
h4 = get_qubit_hamiltonian(mol='h4', geometry=86, basis='sto3g', qubit_transf=qubit_transf)
print(h4)

-0.6385967375551568 [] +
-0.03496705452855378 [X0 X1 Y2 Y3] +
-0.032724726619351434 [X0 X1 Y4 Y5] +
-0.02088944311879026 [X0 X1 Y6 Y7] +
0.03496705452855378 [X0 Y1 Y2 X3] +
0.032724726619351434 [X0 Y1 Y4 X5] +
0.02088944311879026 [X0 Y1 Y6 X7] +
-0.01400483672051457 [X0 Z1 X2 X4 Z5 X6] +
-0.0020414179181859338 [X0 Z1 X2 Y4 Z5 Y6] +
-0.035307899968521766 [X0 Z1 X2 X5 Z6 X7] +
-0.035307899968521766 [X0 Z1 X2 Y5 Z6 Y7] +
-0.011963418802328638 [X0 Z1 Y2 Y4 Z5 X6] +
0.03326648205033583 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
0.021303063248007195 [X0 Z1 Z2 X3 X5 X6] +
-0.03326648205033583 [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
0.021303063248007195 [X0 Z1 Z2 Y3 Y5 X6] +
0.03496705452855378 [Y0 X1 X2 Y3] +
0.032724726619351434 [Y0 X1 X4 Y5] +
0.02088944311879026 [Y0 X1 X6 Y7] +
-0.03496705452855378 [Y0 Y1 X2 X3] +
-0.032724726619351434 [Y0 Y1 X4 X5] +
-0.02088944311879026 [Y0 Y1 X6 X7] +
-0.011963418802328638 [Y0 Z1 X2 X4 Z5 Y6] +
-0.0020414179181859338 [Y0 Z1 Y2 X4 Z5 X6] +
-0.01400483672051457 [Y0 Z1 Y2 Y4 Z5 Y6] 

In [13]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h4, n_spin_orbitals=8, n_electrons=4, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.6385967375551568 [] +
0.043473035527959225 [X0 X1 Z2 Z3] +
-0.04260612649601439 [X0 Y1 Y2] +
-0.023926837604657275 [X0 Y1 Y3] +
-0.06993410905710756 [X0 Z1 Z2] +
-0.06653296410067167 [X0 Z1 Z2 X3] +
-0.0729087454112521 [X0 Y2 Y3] +
0.06993410905710756 [X0 Z2 Z3] +
0.04260612649601439 [Y0 X1 Y2] +
0.02800967344102914 [Y0 X1 Y3] +
0.043473035527959225 [Y0 Y1 Z2 Z3] +
-0.0040828358363718675 [Y0 Y1 X3] +
0.07061579993704353 [Y0 Z1 Y2 Z3] +
-0.06653296410067167 [Y0 Z1 Z2 Y3] +
-0.07061579993704353 [Y0 Y2] +
0.0729087454112521 [Y0 Y2 X3] +
0.01998268392788931 [Z0] +
-0.06544945323870287 [Z0 X1 Z2] +
0.07061579993704353 [Z0 X1 Z2 X3] +
-0.06653296410067167 [Z0 Y1 Y2 Z3] +
0.07061579993704353 [Z0 Y1 Z2 Y3] +
0.38375583026349336 [Z0 Z1] +
0.0040828358363718675 [Z0 Z1 X2] +
0.04177888623758052 [Z0 Z1 Y2 Y3] +
0.1893707647308015 [Z0 Z1 Z2] +
0.4690077520290331 [Z0 Z1 Z2 Z3] +
0.18937076473080144 [Z0 Z1 Z3] +
-0.02800967344102914 [Z0 X2 Z3] +
0.47558802237695896 [Z0 

In [14]:
print("The ground state energy:")
obtain_PES('h4', [86], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
hamiltonian = taper_hamiltonian(h4, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf)
pauli_list = []
interaction_list = []
max_orbitals = 0
#Generate pauli operations list and coefficient list(interactions)
for pauli, interaction in hamiltonian.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)

#Identify Number Max Oribitals    
for x in list(hamiltonian.terms.keys()):
    _max = 0
    for y in x:
        if y[0] > _max:
            _max = y[0]
max_orbitals = _max + 1


h4_matrix = hamiltonianQ(pauli_list, interaction_list, max_orbitals)

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h4_matrix)

print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The ground state energy:
E = -1.9809876600812353 Eh

The eigenvalues in the effective Hamiltonian: 
 [-4.38590669 -3.78595599 -3.76265062 -3.46761965 -2.80416317 -2.50191993
 -2.49040047 -1.86278942  0.57510155  1.37244182  1.399504    1.445074
  2.26844341  2.30516307  2.34273245  3.13539781]


# H20

In [26]:
#########Assumptions made
#Made the assumption based on energy graphs from last notebook
#The hamiltonian outputs the number qubits needed to represent the molecule 
#mapping fermonic states to qubit states --> Qubit representation of hamiltonian
#Jordon-Wigner --> Fermion to Qubit Mapping
qubit_transf = 'jw' # Jordan-Wigner transformations
h2o = get_qubit_hamiltonian(mol='h2o', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2o)

-46.577441376239065 [] +
-0.014754064435548013 [X0 X1 Y2 Y3] +
0.005507932036998482 [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
-0.008605981447809241 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
0.005507932036998482 [X0 X1 X3 Z4 Z5 X6] +
-0.008605981447809241 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.0026730909100339986 [X0 X1 Y4 Y5] +
-0.003753790543621196 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Y13] +
-0.003753790543621196 [X0 X1 X5 Z6 Z7 Z8 Z9 Z10 Z11 X12] +
-0.00669893609969373 [X0 X1 Y6 Y7] +
-9.234301007401337e-05 [X0 X1 Y6 Z7 Z8 Z9 Z10 Y11] +
-9.234301007401335e-05 [X0 X1 X7 Z8 Z9 X10] +
-0.006505428251122435 [X0 X1 Y8 Y9] +
-0.007380770580843633 [X0 X1 Y10 Y11] +
-0.005283643032839696 [X0 X1 Y12 Y13] +
0.014754064435548013 [X0 Y1 Y2 X3] +
-0.005507932036998482 [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
0.008605981447809241 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.005507932036998482 [X0 Y1 Y3 Z4 Z5 X6] +
-0.008605981447809241 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.0026730909100339986 [X0 Y1 Y4 X5] +
0.0037537905436211

In [27]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2o, n_spin_orbitals=14, n_electrons=17, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -46.3574016028953 [] +
0.10505651008351931 [X0] +
-0.011299006073352361 [X0 X1 X2 Z3 Z4 Z6 Z7 X8] +
0.035225593768224865 [X0 X1 X2 Z8 Z9] +
0.0021883512242770174 [X0 X1 Z2 Z3 X4 Z5 Z7 Z8] +
-0.0016647434988545067 [X0 X1 Z2 Z3 Z5 Z6 X7 Z8] +
-0.031728759707490346 [X0 X1 Y3 Y4] +
-0.005996185758108589 [X0 X1 Y3 Z4 Z6 Y7] +
0.011299006073352361 [X0 X1 Z3 Z4 Z6 Z7 X9] +
-0.005996185758108589 [X0 X1 X4 X6] +
0.0021883512242770174 [X0 X1 X4 Z6 Z7 Z8 Z9] +
0.03654446580389839 [X0 X1 X5] +
-0.02482949020614589 [X0 X1 Y6 Y7] +
-0.0016647434988545067 [X0 X1 X7 Z8 Z9] +
-0.015996127256196643 [X0 X1 Y8 Y9] +
-0.011299006073352361 [X0 Y1 Y2 Z3 Z4 Z6 Z7 X8] +
0.035225593768224865 [X0 Y1 Y2 Z8 Z9] +
-0.0021883512242770174 [X0 Y1 Z2 Y3 Z4 Z5 Z7 Z8] +
0.0021883512242770174 [X0 Y1 Z2 Z3 Y4 Z5 Z7 Z8] +
0.0016647434988545067 [X0 Y1 Z2 Z3 Z5 Y6 Z7 Z8] +
-0.0016647434988545067 [X0 Y1 Z2 Z3 Z5 Z6 Y7 Z8] +
0.031728759707490346 [X0 Y1 Y3 X4] +
0.005996185758108589 [X0 Y1 Y3 Z4 Z6 X7

In [28]:
print("The ground state energy:")
obtain_PES('h2o', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
hamiltonian = taper_hamiltonian(h2o, n_spin_orbitals=14, n_electrons=17, qubit_transf=qubit_transf)
pauli_list = []
interaction_list = []
max_orbitals = 0
#Generate pauli operations list and coefficient list(interactions)
for pauli, interaction in hamiltonian.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)

#Identify Number Max Oribitals    
for x in list(hamiltonian.terms.keys()):
    _max = 0
    for y in x:
        if y[0] > _max:
            _max = y[0]
max_orbitals = _max + 1


h2o_matrix = hamiltonianQ(pauli_list, interaction_list, max_orbitals)

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2o_matrix)

print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The ground state energy:
E = -75.01768869621627 Eh

The eigenvalues in the effective Hamiltonian: 
 [-99.23749418 -99.23749418 -99.23620224 ...   6.46742863   6.46831604
   6.46831604]
